<a href="https://colab.research.google.com/github/ekrombouts/GenCareAI/blob/olympia/notebooks/100_note_generation/100_GenerateAnonymousCareNotes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GenCare AI: Synthetic Healthcare Data Generation

**Author:** Eva Rombouts  
**Date:** 2024-05-12  
**Updated:**  
**Version:** 2.0 (wip)

### Description
This notebook generates synthetic healthcare data for NLP experiments.  
It utilizes OpenAI's models to create a synthetic dataset with progress notes of nursing home clients for use in machine learning and data analysis.  

These notes are “anonymous,” meaning they are not based on specific client profiles or scenarios, unlike the notes generated in later steps.

An example of the resulting [dataset](https://huggingface.co/datasets/ekrombouts/dutch_nursing_home_notes) can be found on HuggingFace.   

I considered using LangChain, but generating multiple completions per request is not straightforward, so I stuck to the OpenAI API.

In [1]:
!pip install GenCareAI
from GenCareAI.GenCareAIUtils import GenCareAISetup

setup = GenCareAISetup()

if setup.environment == 'Colab':
        !pip install -q langchain langchain_openai

verbose = True

In [2]:
import pandas as pd

from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List
from langchain_community.callbacks import get_openai_callback

In [3]:
# Constants / variables
seed = 6
# model = 'gpt-4o-mini-2024-07-18'
model = 'gpt-3.5-turbo-0125'
temp = 1.1

num_notes = 10 #Number of notes generated per completion
num_completions = 10 #Number of completions per query

fn = setup.get_file_path('data/gcai_notes.csv')

if verbose:
    print(fn)

/Users/eva/Library/Mobile Documents/com~apple~CloudDocs/Documents/@DataScience/Python/GenCareAI/data/gcai_notes.csv


In [4]:
# Define a Pydantic model for the note structure
class Note(BaseModel):
    note: List[str] = Field(description="rapportage tekst")

# Initialize the output parser using the Pydantic model
output_parser = PydanticOutputParser(pydantic_object=Note)
# Get format instructions for the LLM to output data compatible with the parser
format_instructions = output_parser.get_format_instructions()

if verbose:
    print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"note": {"description": "rapportage tekst", "items": {"type": "string"}, "title": "Note", "type": "array"}}, "required": ["note"]}
```


In [5]:
# Define the prompt template with placeholders for variables
template = """Help me bij het schrijven van fictieve zorgrapportages.
Je spreekt de taal van een niveau 3 zorgmedewerker (Verzorgende IG).
De rapportages gaan over mensen die zijn opgenomen in het verpleeghuis. Deze clienten zijn ernstig beperkt, meestal vanweze een dementie, soms vanwege somatisch onderliggend lijden.

Verzin {num_notes} rapportages over {category}.
Varieer in zinsopbouw, formulering en stijl: gebruik zowel korte als lange zinnen, wissel tussen formeel en informeel taalgebruik, en schrijf vanuit verschillende perspectieven (bijvoorbeeld direct tegen de cliënt of in de derde persoon). Vermijd het noemen van de naam. Zorg ervoor dat sommige rapportages gedetailleerd zijn en andere juist beknopt. Gebruik afwisselende onderwerpen, zoals: {note_topics}.

Voorbeelden:
{examples}
{format_instructions}
"""

In [6]:
# Create a PromptTemplate instance, injecting format instructions as a partial variable
prompt_template = PromptTemplate(
    input_variables=["num_notes", "cat_description", "note_topics", "examples"],
    template=template,
    partial_variables={"format_instructions": format_instructions},
)

# Initialize the language model with specified parameters
llm = ChatOpenAI(api_key=setup.get_openai_key(),model=model, temperature=1.1, n=num_completions)


In [7]:
# List of dictionaries containing input data for each category
input_data_list = [
    {
        "cat": "adl",
        "category": "ADL (Algemene Dagelijkse Levensverrichtingen)",
        "examples": """- Dhr. zijn haar gewassen en zijn baard geschoren.
- Inco van mw, was verzadigd vanmorgen en bed was nat.
- Het is niet goed gegaan Mw had een ongelukje met haar kleding en defeaceren Mw was incontinent Mw geholpen met opfrissen en de kleding in de was gedaan
- U bent vanmorgen gedoucht, uw haren zijn gewassen.
""",
        "note_topics": "wassen, aankleden, tanden poetsen, klaarmaken voor de dag, klaarmaken voor de nacht, douchen, gebitsprothese schoonmaken of hulp na incontinentie.",
    },
    {
        "cat": "eten_drinken",
        "category": "eten en drinken",
        "examples": """- Ik kreeg van de dagdienst door dat dhr. zich verslikt in haar drinken. Drinken verdikt aangeboden. Dit ging goed.
- Ochtendzorg verliep goed, dhr was wel zeer vermoeid. Dhr heeft goed gegeten en gedronken. Dhr is na de lunch op bed geholpen om te rusten.
- Nee ik wil niet meer. ik vond niet lekker. Mw heeft ochtend goed gegeten en gedronken. tussen de middageten mw wilde niet. zij heeft paar hapjes vla gegeten en een glas limonade gedronken.
- Mw heeft op bed een paar hapjes pap gegeten.
- De fresubin creme is niet op voorraad. mw ipv de creme fresubin drink aanbieden Fresubin komt vogende week weer binnen.
""",
        "note_topics": "wat de client wel of niet heeft gegeten, welke hulp nodig is bij eten (volledige hulp, aansporing, aangepast bestek of beker), verslikken, bijhouden vocht- en voedingslijst.",
    },
    {
        "cat": "sociaal",
        "category": "sociale interactie en activiteiten",
        "examples": """- Mw. was goed gestemd vanavond en was heel gezellig aanwezig.
- U keek naar de kerkdienst op buurt 4.
- Dhr zit met verschillende medebewoners in de binnentuin.
- Ik eet samen met mijn dochter. We gaan asperges eten.
- Mw. ging haar gangetje. Ging vanmiddag naar een muziek activiteit.
""",
        "note_topics": "georganiseerde activiteiten, het krijgen van bezoek, bladeren door een tijdschriftje, interactie met medebewoners. Hou er rekening mee dat het gaat over rapportages van mensen in een verpleeghuis, met forse beperkingen, dus de sociale interactie en activiteiten zijn beperkt. Meestal betreft het gezelligheid, maar niet altijd.",
    },
    {
        "cat": "huid",
        "category": "huid en wonden",
        "examples": """- ik heb jeuk op mijn rug dhr behandeld met de cetomacrogol creme
- Wat is dat allemaal? Dhr zat aan het verband om zijn arm te plukken. Wondje op arm is klein. Dhr ervaart het verband onprettig. Pleister op het wondje gedaan.
- Dhr zijn liezen zagen er rustig uit. Dhr zijn scrotum ingesmeerd met licht zinkzalf, deze was wel rood. De liezen met beschermende zalf ingesmeerd.
- Mevr. lijkt nu decubitus te ontwikkelen op haar stuit. Mevr. haar hiel verzorgd, dit zag er oke uit, klein beetje geel beslag. Dit schoongemaakt, daarna verbonden volgens plan Dit in de gaten houden.
""",
        "note_topics": "oedeem, decubituswonden, ontvellingen, roodheid en jeuk van de huid. Te lange nagels, smetplekken.",
    },
    {
        "cat": "medisch_logistiek",
        "category": "medische zorg en familie communicatie",
        "examples": """- Oren van mevr zijn uitgespoten er kwam uit beide oren veel viezigheid.
- Graag Dhr morgen wegen
- Arts vragen voor brutans 5 mg besteld
- Dochter van dhr. belde. Ze gaf aan dat ze een aanbod hebben gekregen voor verblijf in een ander verpleeghuis.
- Fam wil graag een gesprek over bezoek cardioloog in het verleden. Er is iets voorgeschreven, ws doorgegeven aan vorige arts. graag contact met familie opnemen voor gesprek of telefonisch gesprek In artsenvisite bespreken
""",
        "note_topics": "zorgplan besprekingen, kleine medische klachten, verzoeken van familie, bestellen van medicijnen.",
    },
    {
        "cat": "nachten",
        "category": "nachten en slapen",
        "examples": """- Mw. heeft de gehele nacht geslapen
- Mw heeft vannacht niet zo goed geslapen. Mw was veel wakker en wat onrustig. Lastig om mw af te leiden en te zorgen dat mw weer wilde slapen. Mw heeft een slechte nachtrust gehad.
- De sensor is de gehele nacht niet afgegaan bij mw
- Dhr. ging rond 23:30 uur naar bed. Heeft de hele nacht geslapen.
- Dhr. was klaarwakker en wilde uit bed en rammelde aan het bedhek. Dhr. vertelde dat hij opgehaald zou worden. Mw. heeft hem overtuigt om toch te gaan slapen en dhr. luisterde naar mw.
""",
        "note_topics": "onrust en dwalen in de nacht, lekker slapen, toiletgang in de nacht, bellen, scheef in bed liggen.",
    },
    {
        "cat": "onrust",
        "category": "onrust, probleemgedrag, stemming",
        "examples": """- Ga opzij. Wat ben jij lelijk Mw schopte naar een andere bewoner en wilde een ander bewoner slaan. Mw een prikkelarme omgeving aangeboden.
- dhr eet de planten van tafel dhr werd begeleid door collega om het uit te spugen werd hier geagiteerd door.
- Waar is het toilet Mag ik al eten Naar zorg toe lopen, zwaaien naar de zorg om hulp.  Mw vraagt veel bevestiging van de zorg,
- Meneer is wat onrustig loopt jammerend heen en weer en zegt steeds erg moe te zijn. Heeft een trieste blik in zijn ogen. Meneer aangeboden om naar bed te gaan, heeft hier geen rust voor.
""",
        "note_topics": "agitatie, onrust, apathie, verwardheid. Meestal is de verwardheid subtiel, maar soms wat heftiger.",
    },
    {
        "cat": "symptomen",
        "category": "ziekte en symptomen",
        "examples": """- Er zat iets vocht in beide voeten. Dhr had vandaag geen steunkousen aan Blijven observeren
- Urine opvangen is tot nu toe nog niet gelukt(mw heeft er steeds def bij) Vanmiddag ook geen pijn gezien alleen evt wat frustratie als iets niet soepel loopt.
- Ik heb pijn dhr gaf pijn aan aan zijn linker pink en ringvinger. Er zitten daar een soort bloedblaren al wel langer. Graag even in de gaten houden en rapporteren of dhr meer pijn krijgt.
- Dhr. had om 6u zeer veel last van slijm en een vieze smaak in zijn mond. Dhr geassisteerd met het spoelen van zijn mond.
- Erg pijnlijk bij de ADL. Morgen graag overleg met de arts over de pijnmedicatie
- Dhr is erg benauwd, klinkt vol, heeft een reutelende ademhaling.
""",
        "note_topics": "pijn, benauwdheid, misselijkheid, diarree, rugklachten, palliatieve zorg. Meestal zijn de klachten subtiel, maar soms heftiger.",
    },
    {
        "cat": "mobiliteit",
        "category": "mobiliteit en transfers",
        "examples": """- Vandaag geholpen met de passieve lift. Dit ging goed.
- Veel rondgelopen vandaag. Mw vergeet steeds haar rollator.
- De banden van de rolstoel zijn zacht. Kan de fysio hier naar kijken?
- De transfers gaan steeds moeilijker. Mw hangt erg in de actieve lift. Glijdt weg. Wil graag nog met de actieve lift geholpen worden, maar dit gaat eigenlijk niet meer. @ Ergo, graag je advies
""",
        "note_topics": "loophulpmiddelen, de rolstoel, valgevaar, valincidenten, transfers, tilliften. De meeste rapportages gaan over dagelijkse dingetjes, dus niet alles is een ernstig incident.",
    },
]


if verbose:
    sample = input_data_list[0]
    sample_prompt = template.format(
        num_notes=10,
        category=sample['category'],
        note_topics=sample['note_topics'],
        examples=sample['examples'],
        format_instructions=format_instructions
)
    print(sample_prompt)


Help me bij het schrijven van fictieve zorgrapportages.
Je spreekt de taal van een niveau 3 zorgmedewerker (Verzorgende IG).
De rapportages gaan over mensen die zijn opgenomen in het verpleeghuis. Deze clienten zijn ernstig beperkt, meestal vanweze een dementie, soms vanwege somatisch onderliggend lijden.

Verzin 10 rapportages over ADL (Algemene Dagelijkse Levensverrichtingen).
Varieer in zinsopbouw, formulering en stijl: gebruik zowel korte als lange zinnen, wissel tussen formeel en informeel taalgebruik, en schrijf vanuit verschillende perspectieven (bijvoorbeeld direct tegen de cliënt of in de derde persoon). Vermijd het noemen van de naam. Zorg ervoor dat sommige rapportages gedetailleerd zijn en andere juist beknopt. Gebruik afwisselende onderwerpen, zoals: wassen, aankleden, tanden poetsen, klaarmaken voor de dag, klaarmaken voor de nacht, douchen, gebitsprothese schoonmaken of hulp na incontinentie..

Voorbeelden:
- Dhr. zijn haar gewassen en zijn baard geschoren.
- Inco van mw

In [8]:
results = []

# Loop through the input data for each category
for input_data in input_data_list:
    # Add num_notes to the input data
    input_data_with_num_notes = {**input_data, "num_notes": num_notes}

    # Generate the prompt
    prompt = prompt_template.format(**input_data_with_num_notes)

    if verbose:
      print(f"Generating notes for category: {input_data['category']}")

    # Generate responses
    response = llm.generate([[prompt]])

    results.append(response)
      
if verbose:
    print((results[0]).generations[0][0].text)


Generating notes for category: ADL (Algemene Dagelijkse Levensverrichtingen)
Generating notes for category: eten en drinken
Generating notes for category: sociale interactie en activiteiten
Generating notes for category: huid en wonden
Generating notes for category: medische zorg en familie communicatie
Generating notes for category: nachten en slapen
Generating notes for category: onrust, probleemgedrag, stemming
Generating notes for category: ziekte en symptomen
Generating notes for category: mobiliteit en transfers
{
  "note": [
    "Cliënt is vanmorgen geholpen met wassen en aankleden.",
    "Mw heeft zelfstandig haar tanden gepoetst na herinnering van verzorgende.",
    "Dhr. had vanmorgen last van incontinentie, is verschoond en bed is verschoond.",
    "Cliënt had moeite met aan- en uitkleden, daarbij geholpen door verzorgende.",
    "Mw had vanmiddag extra hulp nodig bij toiletgang vanwege obstipatie.",
    "Dhr. wilde vanmorgen niet douchen, dit is uitgesteld tot vanavond.",
 

In [9]:
data = []

# Loop through the results for each category
for i, response in enumerate(results):
    # Loop through the completions for each category
    for j, generation in enumerate(response.generations):
        # Loop through the individual generations within each completion
        for k, gen_text in enumerate(generation):
            generated_text = gen_text.text  # Extract the generated text

            try:
                # Parse the generated text using the Pydantic parser
                parsed_output = output_parser.parse(generated_text)

                # Loop through the parsed notes and add them to the data list
                for note in parsed_output.note:
                    data.append({
                        'category': input_data_list[i]['cat'],  # Add the category from input data
                        'completion': j + 1,  # Track which completion this belongs to
                        'generation': k + 1,  # Track which generation within the completion
                        'note': note  # Store the parsed note text
                    })

            except Exception as e:
                print(f"Error parsing the output for category {i+1}, completion {j+1}, generation {k+1}: {e}")

# Convert the data list into a Pandas DataFrame
df = pd.DataFrame(data)

if verbose:
    print(df.head())

Error parsing the output for category 9, completion 1, generation 7: Invalid json output: {
	"note": [
		- Vandaag is mevrouw met de actieve lift uit bed gehaald. Ze was erg onrustig en probeerde te staan. Samen met een collega is de transfer veilig verlopen. 
		- Meneer weigert de rollator te gebruiken en gaat steeds op pad zonder ondersteuning. Hierdoor is er een verhoogd risico op valincidenten. Graag extra observatie inzetten.
		- De banden van de rolstoel zijn opgepompt, dit zorgt voor meer comfort tijdens het gebruik. Cliënt lijkt nu beter te kunnen sturen.
		- Mw heeft vanochtend met behulp van de passieve lift gedoucht. De transfer verliep soepel en ze genoot zichtbaar van de frisse start van de dag.
		- Tijdens de lunch is mw gevallen bij het overzetten van de rolstoel naar de stoel aan tafel. Geen letsel opgelopen, maar extra voorzichtigheid is nodig.
		- Meneer klaagt over pijn in de benen bij het opstaan uit bed. Wellicht is een andere transfermethode beter geschikt, graag 

In [10]:
data = []

# Loop through the results for each category
for i, response in enumerate(results):
    # Loop through the completions for each category
    for j, generation in enumerate(response.generations):
        # Loop through the individual generations within each completion
        for k, gen_text in enumerate(generation):
            generated_text = gen_text.text  # Extract the generated text
            
            # Schoonmaken van de gegenereerde tekst
            cleaned_output = generated_text.replace("```json", "").replace("```", "").strip()

            try:
                # Parse de schoongemaakte tekst
                parsed_output = output_parser.parse(cleaned_output)

                # Voeg de geparsed notes toe aan de data lijst
                for note in parsed_output.note:
                    data.append({
                        'category': input_data_list[i]['cat'],
                        'generation': k + 1,
                        'note': note
                    })

            except Exception as e:
                # Log de originele output die de fout heeft veroorzaakt
                print(f"Error parsing the output for category {input_data_list[i]['cat']}, generation {k + 1}: {e}")
                print(f"Original output: {cleaned_output}")  # Deze regel is toegevoegd

# Zet de data lijst om in een Pandas DataFrame
df = pd.DataFrame(data)

if verbose:
    print(df.head())

Error parsing the output for category mobiliteit, completion 1, generation 7: Invalid json output: {
	"note": [
		- Vandaag is mevrouw met de actieve lift uit bed gehaald. Ze was erg onrustig en probeerde te staan. Samen met een collega is de transfer veilig verlopen. 
		- Meneer weigert de rollator te gebruiken en gaat steeds op pad zonder ondersteuning. Hierdoor is er een verhoogd risico op valincidenten. Graag extra observatie inzetten.
		- De banden van de rolstoel zijn opgepompt, dit zorgt voor meer comfort tijdens het gebruik. Cliënt lijkt nu beter te kunnen sturen.
		- Mw heeft vanochtend met behulp van de passieve lift gedoucht. De transfer verliep soepel en ze genoot zichtbaar van de frisse start van de dag.
		- Tijdens de lunch is mw gevallen bij het overzetten van de rolstoel naar de stoel aan tafel. Geen letsel opgelopen, maar extra voorzichtigheid is nodig.
		- Meneer klaagt over pijn in de benen bij het opstaan uit bed. Wellicht is een andere transfermethode beter geschik

In [ ]:
df.info()

In [11]:
# Sla de DataFrame op als CSV-bestand
df.to_csv(fn, index=False)

# Bevestig dat het bestand succesvol is opgeslagen, als verbose aan staat
if verbose:
    print(f"Data opgeslagen naar: {fn}")

Data opgeslagen naar /Users/eva/Library/Mobile Documents/com~apple~CloudDocs/Documents/@DataScience/Python/GenCareAI/data/gcai_notes.csv


In [13]:
# !pip -q install datasets

from datasets import Dataset, DatasetDict

dataset = Dataset.from_pandas(df)
# Path for pushing dataset to HuggingFace Hub
path_hf_sampc_long = "ekrombouts/Olympia_notes"
# Commit message for the push
commit_message = "Synthetic notes, v2.0wip"


# Split the dataset into training and test/validation splits
train_testvalid_split = dataset.train_test_split(test_size=0.2, seed=seed)

# Further split the test set into validation and test sets
test_valid_split = train_testvalid_split['test'].train_test_split(test_size=0.5, seed=seed)

# Create a DatasetDict object to hold the splits
dataset_dict = DatasetDict({
    'train': train_testvalid_split['train'],
    'validation': test_valid_split['train'],
    'test': test_valid_split['test'],
})

# Push the dataset to HuggingFace Hub with the specified path and commit message
dataset_dict.push_to_hub(path_hf_sampc_long,
                         commit_message=commit_message,
                         private=True)

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.01s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/ekrombouts/Olympia_notes/commit/c0326aac115d8300ff49fdb566e48fff34e8aaa1', commit_message='Synthetic notes, v2.0wip', commit_description='', oid='c0326aac115d8300ff49fdb566e48fff34e8aaa1', pr_url=None, pr_revision=None, pr_num=None)